In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# applicant_arrival = 0.03
# applicant_departure = 0.03
vacancy_rate = 0.05

In [3]:
def preference_creation(family_size: int):
    if family_size == 1:
        preferences = [2,3]
    elif family_size == 2:
        preferences = [1,3,4]
    elif family_size == 3:
        preferences = [1,2,4]
    elif family_size == 4:
        preferences = [2,3,5]
    elif family_size == 5:
        preferences = [4,6]
    else:
        preferences = [5]
    
    output = [min(family_size, 6)] + random.sample(preferences, len(preferences))
    return output

def housing_vacancy(stock):
    
    # Probability weights for each housing type (reverse proportion to the key size)
    weights = [1 / key for key in stock]

    
    # Distribute 80 units (~10%) randomly across types with weighted choice
    units_to_add = 80 # int(vacancy_rate * 817.0)
    
    while units_to_add > 0:
        random_type = random.choices(list(stock.keys()), weights=weights)[0]
        add_count = random.randint(1, units_to_add)  # Random count to add

        # Increment the randomly chosen type by the add_count
        stock[random_type] += add_count

        units_to_add -= add_count  # Reduce the remaining units to add

    return stock


In [4]:
# import file
df = pd.read_csv("Eligible_Applicants.csv")

# https://www.habc.org/habc-information/about-us/news/habc-reopens-public-housing-waiting-list/

# filtering eligible applicants 
df = df[df['familysize'] != 0] # removing applicants with family size = 0
df = df[df['income'] != 0] # removing applicants with income = 0
df = df[df['race'] != 0]
df = df.drop(df.columns[0:1], axis = 1)
df = df[~(df['income'] > 14160 + (7710 * df['familysize']))]
# df = df.loc[df.index.repeat(20)].reset_index(drop=True)


preferences = []
for index, row in df.iterrows():
    preferences.append(preference_creation(int(row['familysize'])))

df['preferences'] = preferences 

# saving original
safe = df
print(f"Total eligible applicants: {len(df)}")

Total eligible applicants: 3473


In [5]:
# housing unit stock
housing_units = {
    1: 9820,
    2: 4349,
    3: 3107,
    4: 1788,
    5: 880,
    6: 569
}

# set initial vacancy rate

for key in housing_units.keys():
    housing_units[key] = int(housing_units[key] * vacancy_rate)
print(f"Total Housing Units Available: {sum(housing_units.values())}")

Total Housing Units Available: 1024


In [6]:
priorities = ['income', 'numchildren', 'numelderly', 'veteran', 'disability']

# For those on the waitlist, find matching, vacant units

for priority in priorities:
    # reset dataframe to infinite wait times and ranking = 0
    df = safe

    # sort dataframe by priority system and save to waitlist
    if priority == 'income':
        waitlist = df.sort_values(by=priority, ascending=True)
    else:
        waitlist = df.sort_values(by=priority, ascending=False)

    # maximum 12*5 periods (once a month for 5 years), end when waitlist is empty
    period = 12*5
    for _ in range(int(period)):

        # break when waitlist is empty
        if waitlist.shape[0] == 0:
            break
        # add x number of housing units per cycle
        housing_units = housing_vacancy(housing_units)

        # for each applicant in the wait list
        for _, applicant in waitlist.iterrows():

            # identify the serial number
            serial_no = applicant['serialno']

            # for each choice in the preferences list
            for rank in applicant['preferences']:

                # if that housing size/type is available
                if housing_units[rank] > 0:

                    # set that housing choice to be their "ranking" (the preference they are matched with)
                    df.loc[df['serialno'] == serial_no, 'ranking'] = applicant['preferences'].index(rank) + 1

                    # reduce that available housing type by 1
                    housing_units[rank] -= 1

                    # remove them from the waitlist
                    waitlist = waitlist[waitlist['serialno'] != serial_no]
                    break
        
        # for all umatched, add 1 to waittime
        df.loc[df['ranking'] == 0, 'waittime'] += 1
    
    # save df
    filename = f'results/dfs/trial_3/housing_allocation_{priority}.csv'
    df.to_csv(filename, index = False)

    # Group by 'Race' and calculate the percentage of top choice within each group
    result = df.groupby('race').agg(
        Percentage=('ranking', lambda x: (x == 1).sum() / len(x) * 100),
        AverageWaitTime=('waittime', 'mean')
    ).reset_index()

    # Round 'Percentage' to 2 decimal points and format as percentage
    result['Percentage'] = result['Percentage'].map('{:.2f}%'.format)

    result.columns = ['Race', 'Percentage of Top Choice', 'Average Wait Time']

    result.to_csv(f'results/stats/trial_3/housing_statistics_{priority}.csv', index=False)

